<p style="font-weight:bold;"> <span style="font-size: 36px"> Scenario Yield Curve Import Test </span> </p>

In [0]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

In [0]:
DataSource.Reset(x => x.ResetType<RawVariable>().ResetType<IfrsVariable>().ResetType<YieldCurve>());

# Setting Constants

In [0]:
var ws = Workspace.CreateNew();
ws.InitializeFrom(DataSource);

In [0]:
var argsBestEstimate = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, null, ImportFormats.Cashflow);
var argsScenarioYieldCurve = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, "YCUP1.0pct", ImportFormats.Cashflow);
var argsScenarioTransactionalData = new ImportArgs ("CH", 2020, 12, Periodicity.Quarterly, "SRUP1.0pct", ImportFormats.Cashflow);

In [0]:
var initStorage = new ImportStorage(argsBestEstimate, DataSource, ws);
await initStorage.InitializeAsync();

In [0]:
var actualEstimateTypes = initStorage.EstimateTypesByImportFormat[ImportFormats.Actual];
var cashflowEstimateTypes = initStorage.EstimateTypesByImportFormat[ImportFormats.Cashflow];
var actualAndCashflowEstimateTypes = actualEstimateTypes.Intersect(cashflowEstimateTypes);
var onlyActualEstimateTypes = actualEstimateTypes.Except(cashflowEstimateTypes);
var onlyCashflowEstimateTypes = cashflowEstimateTypes.Except(actualEstimateTypes);

# Best Estimate Initialization

In [0]:
var basicYieldCurve = @"
@@Main
Year,Month,Scenario
2020,1,
@@YieldCurve
Currency,Values0,Values1,Values2,Values3
USD,0.002,0.002,0.002,0.002";

In [0]:
await Import.FromString(basicYieldCurve).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync()

In [0]:
var cashflowBestEstimateInit = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,100,0,0,100,0,0,100,0,0,100,0,0,0,100,0,0,100,0,0,100,0,0,100,0
DT1.1,NIC,BE,CL,C,,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25,-25
DT1.1,,CU,CL,C,,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-10,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-3
DT1.1,,RA,CL,C,,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5,-2.5
DTR1.1,PR,BE,CL,C,,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0,50,0,0
DTR1.1,NIC,BE,CL,C,,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5,12.5
DTR1.1,,CU,CL,C,,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5,-5
DTR1.1,,RA,CL,C,,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25,1.25";

In [0]:
var actualsBestEstimateInit = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,
@@Actual
DataNode,AocType,AmountType,EstimateType,AccidentYear,Value
DT1.1,CF,ACA,A,,-10
DT1.1,CF,AEA,A,,-5
DT1.1,CF,NIC,A,,-280
DT1.1,CF,PR,A,,400
DTR1.1,CF,PR,A,,-200
DTR1.1,CF,NIC,A,,140";

In [0]:
await Import.FromString(cashflowBestEstimateInit).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await Import.FromString(actualsBestEstimateInit).WithFormat(ImportFormats.Actual).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateInit = await DataSource.Query<IfrsVariable>().ToArrayAsync();

# Scenario Yield Curve Import

In [0]:
var yieldCurveScenario = @"
@@Main
Year,Month,Scenario
2020,12,YCUP1.0pct
@@YieldCurve
Currency,Values0,Values1,Values2,Values3
EUR,0.108685495,0.108685495,0.118498471,0.119966839
CHF,0.102,0.102,0.102,0.102
USD,0.102,0.102,0.102,0.102";

In [0]:
await Import.FromString(yieldCurveScenario).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioYieldCurve);
var ivsScenarioYieldCurve = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsScenarioYieldCurve.Where(x => Math.Abs(x.Value) > Precision).Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());
var actualVars = diffs.Where(x => onlyActualEstimateTypes.Contains(x.EstimateType)).ToArray();
var cashflowVars = diffs.Where(x => onlyCashflowEstimateTypes.Contains(x.EstimateType)).ToArray();

In [0]:
actualVars.Length.Should().Be(0);
cashflowVars.Length.Should().NotBe(0);

# Re-Import of the same Yield Curve does not trigger IFRS17 Calculations

In [0]:
await Import.FromString(yieldCurveScenario).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioYieldCurve);
var ivsScenarioYieldCurveReimport = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsScenarioYieldCurve.Except(ivsScenarioYieldCurve, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().Be(0);

# Multiple Scenarios 

In this example, a new Cashflow for a different Scenario - SRUP1.0pct - is imported. 
The aim is to check how the Yield Curve importer behaves when a new Best Estimate Yield Curve is imported. 
We can identify the following two cases: 
 - a Scenario including a dedicated Yield Curve: only the Best Estimate variables are re-calculated;
 - a Scenario including only transactional data: re-calculation is needed to account for the updated Best Estimate Yield Curve. 

In [0]:
var cashflowScenario = @"
@@Main
ReportingNode,Year,Month,Scenario
CH,2020,12,SRUP1.0pct
@@Cashflow
DataNode,AmountType,EstimateType,AocType,Novelty,AccidentYear,Values0,Values1,Values2,Values3,Values4,Values5,Values6,Values7,Values8,Values9,Values10,Values11,Values12,Values13,Values14,Values15,Values16,Values17,Values18,Values19,Values20,Values21,Values22,Values23
DT1.1,PR,BE,CL,C,,110,0,0,110,0,0,110,0,0,110,0,0,0,110,0,0,110,0,0,110,0,0,110,0
DT1.1,NIC,BE,CL,C,,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5,-27.5
DT1.1,,RA,CL,C,,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75,-2.75";

In [0]:
await Import.FromString(cashflowScenario).WithFormat(ImportFormats.Cashflow).WithTarget(DataSource).ExecuteAsync()

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioTransactionalData);
var ivsScenarioTransactionalData = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var bestEstimateYieldCurve = @"
@@Main
Year,Month
2020,12
@@YieldCurve
Currency,Values0,Values1,Values2,Values3
EUR,0.108,0.108,0.118,0.119
CHF,0.103,0.103,0.103,0.103
USD,0.903,0.103,0.103,0.103";

In [0]:
await Import.FromString(bestEstimateYieldCurve).WithFormat(ImportFormats.YieldCurve).WithTarget(DataSource).ExecuteAsync()

## Best Estimate variables are re-calculated

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsBestEstimate);
var ivsBestEstimateUpdated = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsBestEstimateUpdated.Except(ivsBestEstimateInit, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().NotBe(0);

## Scenario with dedicated Yield Curve does not change

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioYieldCurve);
var ivsScenarioYieldCurveUpdated = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsScenarioYieldCurveUpdated.Where(x => Math.Abs(x.Value) > Precision).Except(ivsScenarioYieldCurve, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().Be(0);

## Scenario including Transactional Data solely is re-calculated

In [0]:
await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(argsScenarioTransactionalData);
var ivsScenarioTransactionalDataUpdated = await DataSource.Query<IfrsVariable>().ToArrayAsync();

In [0]:
var diffs = ivsScenarioTransactionalDataUpdated.Where(x => Math.Abs(x.Value) > Precision).Except(ivsScenarioTransactionalData, IfrsVariableComparer.Instance());

In [0]:
diffs.Count().Should().NotBe(0);